In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras_tuner import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError,BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.activations import relu, elu, sigmoid, softmax,linear
from sklearn.model_selection import train_test_split


2024-05-06 17:31:51.732100: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-06 17:31:51.732384: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-06 17:31:51.734509: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-06 17:31:51.762737: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-06 17:31:52.303067: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
data = pd.read_csv(r'/home/ahmed/Ai/Neural-Networks-tutorial/data/Real_Combine.csv')
data.dropna(inplace=True)  
X, Y = data.iloc[:, :-1], data.iloc[:, -1] 
data

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833
...,...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9,288.416667
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2,256.833333
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8,169.000000
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5,186.041667


In [74]:
def build_model(hyperparameters):
    model = Sequential()  # The Sequential model, which i will use to build the network

    for i in range(
        hyperparameters.Int("number_of_hidden_layers", min_value=2, max_value=20)
    ):  # which means that i will have a number of hidden layers between 2 and 20
        model.add(
            Dense(
                units=hyperparameters.Int(
                    "neurons_of_layer_" + str(i), min_value=16, max_value=256, step=16
                ),  # The number of neurons in each layer will be between 16 and 256)
                activation=hyperparameters.Choice(
                    "activation_of_layer_" + str(i),
                    values=['relu', 'softmax', 'linear'],
                ),
            )  # The activation function of each layer will be one of the following functions: relu, elu, sigmoid, softmax, linear
        )
        model.add(Dropout(hyperparameters.Float("dropout_of_layer_" + str(i), 0, 0.5)))
    model.add(Dense(1, activation='linear'))

    model.compile(
        optimizer=Adam(
            learning_rate=hyperparameters.Choice(
                "learning_rate",
                values=[1e-2, 1e-3, 1e-4],
            )
        ),
        loss=MeanSquaredError(),
        metrics=["accuracy"],
    )
    return model

In [75]:
tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=3,
    directory="hyperparameters", # for saving the logs and checkpoints
)

In [76]:
tuner.search_space_summary()

Search space summary
Default search space size: 8
number_of_hidden_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
neurons_of_layer_0 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 256, 'step': 16, 'sampling': 'linear'}
activation_of_layer_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'softmax', 'linear'], 'ordered': False}
dropout_of_layer_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': None, 'sampling': 'linear'}
neurons_of_layer_1 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 256, 'step': 16, 'sampling': 'linear'}
activation_of_layer_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'softmax', 'linear'], 'ordered': False}
dropout_of_layer_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': None, 'sampling': 'linear'}
learning_rate (Choice)
{'default':

In [77]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((764, 8), (328, 8), (764,), (328,))

In [78]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 10s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.04268292710185051
Total elapsed time: 00h 00m 37s


In [79]:
tuner.results_summary()

Results summary
Results in hyperparameters/untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
number_of_hidden_layers: 10
neurons_of_layer_0: 208
activation_of_layer_0: linear
dropout_of_layer_0: 0.3508364888783842
neurons_of_layer_1: 176
activation_of_layer_1: linear
dropout_of_layer_1: 0.2689344564475333
learning_rate: 0.001
neurons_of_layer_2: 224
activation_of_layer_2: softmax
dropout_of_layer_2: 0.43035407076300547
neurons_of_layer_3: 208
activation_of_layer_3: relu
dropout_of_layer_3: 0.2687993395402663
neurons_of_layer_4: 80
activation_of_layer_4: linear
dropout_of_layer_4: 0.12975247738834494
neurons_of_layer_5: 16
activation_of_layer_5: linear
dropout_of_layer_5: 0.13146476384411687
neurons_of_layer_6: 32
activation_of_layer_6: relu
dropout_of_layer_6: 0.13510299071710108
neurons_of_layer_7: 96
activation_of_layer_7: linear
dropout_of_layer_7: 0.24073043470922068
neurons_of_layer_8: 144
activation_of_layer_

In [80]:
tuner.get_best_hyperparameters()[0].values

{'number_of_hidden_layers': 10,
 'neurons_of_layer_0': 208,
 'activation_of_layer_0': 'linear',
 'dropout_of_layer_0': 0.3508364888783842,
 'neurons_of_layer_1': 176,
 'activation_of_layer_1': 'linear',
 'dropout_of_layer_1': 0.2689344564475333,
 'learning_rate': 0.001,
 'neurons_of_layer_2': 224,
 'activation_of_layer_2': 'softmax',
 'dropout_of_layer_2': 0.43035407076300547,
 'neurons_of_layer_3': 208,
 'activation_of_layer_3': 'relu',
 'dropout_of_layer_3': 0.2687993395402663,
 'neurons_of_layer_4': 80,
 'activation_of_layer_4': 'linear',
 'dropout_of_layer_4': 0.12975247738834494,
 'neurons_of_layer_5': 16,
 'activation_of_layer_5': 'linear',
 'dropout_of_layer_5': 0.13146476384411687,
 'neurons_of_layer_6': 32,
 'activation_of_layer_6': 'relu',
 'dropout_of_layer_6': 0.13510299071710108,
 'neurons_of_layer_7': 96,
 'activation_of_layer_7': 'linear',
 'dropout_of_layer_7': 0.24073043470922068,
 'neurons_of_layer_8': 144,
 'activation_of_layer_8': 'relu',
 'dropout_of_layer_8': 0.32

In [81]:
final_model = tuner.get_best_models()[0]
final_model.summary()

/home/ahmed/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 208)            │         1,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 208)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 176)            │        36,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 176)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 224)            │        39,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 208)            │        46,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 208)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 80)             │        16,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 16)             │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 96)             │         3,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 144)            │        13,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 144)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 192)            │        27,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │           193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 188,833 (737.63 KB)

 Trainable params: 188,833 (737.63 KB)

 Non-trainable params: 0 (0.00 B)